## Pyspark

In [1]:
from pyspark.sql import SparkSession
import findspark
findspark.init()

Spark = SparkSession.builder.master("local[*]").appName("SparkByExamples.com").getOrCreate()

Py4JError: An error occurred while calling None.org.apache.spark.sql.SparkSession. Trace:
py4j.Py4JException: Constructor org.apache.spark.sql.SparkSession([class org.apache.spark.SparkContext, class java.util.HashMap]) does not exist
	at py4j.reflection.ReflectionEngine.getConstructor(ReflectionEngine.java:179)
	at py4j.reflection.ReflectionEngine.getConstructor(ReflectionEngine.java:196)
	at py4j.Gateway.invoke(Gateway.java:237)
	at py4j.commands.ConstructorCommand.invokeConstructor(ConstructorCommand.java:80)
	at py4j.commands.ConstructorCommand.execute(ConstructorCommand.java:69)
	at py4j.ClientServerConnection.waitForCommands(ClientServerConnection.java:182)
	at py4j.ClientServerConnection.run(ClientServerConnection.java:106)
	at java.base/java.lang.Thread.run(Thread.java:834)



## Pytorch

In [1]:
import torch
torch.cuda.is_available()


if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
    
print("using", device, "device")

using cuda device


In [5]:
import time

matrix_size = 32*600

x = torch.randn(matrix_size, matrix_size)
y = torch.randn(matrix_size, matrix_size)

print("************ CPU SPEED ***************")
start = time.time()
result = torch.matmul(x,y)
print(time.time() - start)
print("verify device:", result.device)

x_gpu = x.to(device)
y_gpu = y.to(device)
torch.cuda.synchronize()

for i in range(3):
    print("************ GPU SPEED ***************")
    start = time.time()
    result_gpu = torch.matmul(x_gpu,y_gpu)
    torch.cuda.synchronize()
    print(time.time() - start)
    print("verify device:", result_gpu.device)


************ CPU SPEED ***************
59.71805810928345
verify device: cpu
************ GPU SPEED ***************
1.7862207889556885
verify device: cuda:0
************ GPU SPEED ***************
1.81290864944458
verify device: cuda:0
************ GPU SPEED ***************
1.8004319667816162
verify device: cuda:0


In [6]:
result_gpu

tensor([[-142.4877, -157.4479, -134.2422,  ...,   62.2809,    0.5038,
            7.5595],
        [ 219.8301,  136.9860,  201.6911,  ...,  -40.5193,  -12.2883,
          152.4540],
        [  50.6536, -106.3733, -110.4533,  ...,   23.4575,   70.9110,
           31.7738],
        ...,
        [  -2.7893, -232.6013, -174.9835,  ...,   35.7527,  -63.3191,
          -63.1375],
        [-144.2040,   39.6278,  128.2334,  ...,  -52.6592, -101.1328,
           -3.8151],
        [-129.5593, -190.7631,  130.5468,  ..., -249.3604, -185.3102,
          142.8233]], device='cuda:0')

In [7]:
result

tensor([[-142.4874, -157.4482, -134.2427,  ...,   62.2808,    0.5036,
            7.5589],
        [ 219.8308,  136.9859,  201.6908,  ...,  -40.5193,  -12.2881,
          152.4539],
        [  50.6536, -106.3734, -110.4541,  ...,   23.4573,   70.9108,
           31.7742],
        ...,
        [  -2.7895, -232.6006, -174.9838,  ...,   35.7528,  -63.3191,
          -63.1375],
        [-144.2037,   39.6275,  128.2333,  ...,  -52.6593, -101.1328,
           -3.8151],
        [-129.5596, -190.7642,  130.5474,  ..., -249.3604, -185.3098,
          142.8232]])

## Tensorflow 

In [9]:
import tensorflow as tf
import tensorflow_datasets as tfds
print("TensorFLow version:",tf.__version__)
print("Num GPUs Available:", len(tf.config.experimental.list_physical_devices('GPU')))
print("Num CPUs Available:", len(tf.config.experimental.list_physical_devices('CPU')))



TensorFLow version: 2.10.1
Num GPUs Available: 1
Num CPUs Available: 1


In [10]:
from tensorflow.python.client import device_lib
def get_available_devices():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos]
print(get_available_devices()) 

['/device:CPU:0', '/device:GPU:0']


In [11]:
tf.test.is_built_with_cuda()

True

In [2]:
%%time
import tensorflow as tf
import tensorflow_datasets as tfds
print("TensorFlow version:", tf.__version__)
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
tf.config.list_physical_devices('GPU')
(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)
def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label
batch_size = 128
ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(batch_size)
ds_train = ds_train.prefetch(tf.data.experimental.AUTOTUNE)
ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_test = ds_test.batch(batch_size)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.experimental.AUTOTUNE)
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(32, kernel_size=(3, 3),
                 activation='relu'),
  tf.keras.layers.Conv2D(64, kernel_size=(3, 3),
                 activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
#   tf.keras.layers.Dropout(0.25),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
#   tf.keras.layers.Dropout(0.5),
  tf.keras.layers.Dense(10, activation='softmax')
])
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(0.001),
    metrics=['accuracy'],
)
model.fit(
    ds_train,
    epochs=12,
    validation_data=ds_test,
)

TensorFlow version: 2.10.1
Num GPUs Available:  1
Cause: Unable to locate the source code of <function normalize_img at 0x0000026813CDF490>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: Unable to locate the source code of <function normalize_img at 0x0000026813CDF490>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: Unable to locate the source code of <function normalize_img at 0x0000026813CDF490>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Epoch 1/12
469/469 [==============================] - 4s 5ms/step - loss: 0.1490 - accuracy: 0.9546 - val_loss: 0.0662 - val_accuracy: 0.9774
Epoch 2/12
469/469 [==============================] - 2s 5ms/step - loss: 0.0427 - accuracy: 0.9872 - val_loss: 0.0379 - val_accuracy: 0.9875
Epoch 3/12
469/469 [==============================] - 2s 5ms/step - loss: 0.0264 - accuracy: 0.9918 - val_loss: 0.0402 - val_accuracy: 0.9860
Epoch 4/12
469/469 [============================